In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
import sys
import numpy as np
import scipy.misc
import os
os.environ.setdefault('CUDA_VISIBLE_DEVICES','1')

In [ ]:
def linear(input, output_dim, scope=None, stddev=0.01):
    norm = tf.random_normal_initializer(stddev=stddev)
    const = tf.constant_initializer(0.0)
    with tf.variable_scope(scope or 'linear'):
        w = tf.get_variable('w', [input.get_shape()[1], output_dim], initializer=norm)
        b = tf.get_variable('b', [output_dim], initializer=const)
        return tf.matmul(input, w) + b

In [ ]:
# Create the model
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

hidden  = tf.nn.relu(linear(y_,  1000, scope='c_h1', stddev=0.01))
a1 = tf.nn.relu(linear(hidden,  1000, scope='c_a1', stddev=0.01))
a2 = tf.nn.relu(linear(hidden,  1000, scope='c_a2', stddev=0.01))
a3 = tf.nn.relu(linear(hidden,  1000, scope='c_a3', stddev=0.01))
b1 = tf.nn.relu(linear(hidden,  1000, scope='c_b1', stddev=0.01))
b2 = tf.nn.relu(linear(hidden,  1000, scope='c_b2', stddev=0.01))
b3 = tf.nn.relu(linear(hidden,  1000, scope='c_b3', stddev=0.01))



h1 =linear(x,  1000, scope='c_layer1', stddev=0.01)
h1_c = tf.nn.relu(tf.multiply(h1,a1)+b1)
h2 = tf.nn.relu(linear(h1_c, 1000, scope='c_layer2', stddev=0.01))
h2_c = tf.nn.relu(tf.multiply(h2,a2)+b2)
h3 = tf.nn.relu(linear(h2_c, 1000, scope='c_layer3', stddev=0.01))
h3_c = tf.nn.relu(tf.multiply(h3,a3)+b3)
h4 = linear(h3_c, 784, scope='c_layer4', stddev=0.01)
out_vecter = tf.sigmoid(h4)
out_image = tf.reshape(out_vecter,[-1,28,28])

def recognition_network(images,reuse = False):
    if reuse:
            tf.get_variable_scope().reuse_variables()
    output = tf.nn.relu(linear(images, 1000, scope='r_layer1', stddev=0.01))
    output = tf.nn.softmax(linear(output, 10, scope='r_layer2', stddev=0.01))
    return output

# Define loss and optimizer
y_clf = recognition_network(x)
y_clf_g = recognition_network(out_vecter,reuse =True)

cross_entropy_recog = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(None,y_,y_clf))
cross_entropy_gen = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(None, y_,y_clf_g)) 
# + \
# tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_clf_g, y_))
reconstruction_loss = tf.reduce_mean(tf.squared_difference(out_vecter,x))
loss = reconstruction_loss + 7.2*cross_entropy_gen
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
#train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
#t_vars = tf.trainable_variables()
t_vars = tf.trainable_variables()
contr_params = [var for var in t_vars if 'c_' in var.name]
recog_params = [var for var in t_vars if 'r_' in var.name]

optim = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss,var_list=contr_params) 
optim_recog = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cross_entropy_recog,var_list=recog_params) 

session_conf = tf.ConfigProto(gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5),
                              allow_soft_placement=True,
                              log_device_placement=False)
sess = tf.InteractiveSession(config=session_conf)
# Train
tf.initialize_all_variables().run()
batch_size = 100
#pretrain recognition network
# for step in range(5001):
#     batch_xs, batch_ys = mnist.train.next_batch(batch_size)
#     _ = sess.run(optim_recog,feed_dict={x: batch_xs,y_:batch_ys})

for step in range(3001):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    norm_image = np.random.uniform(0, 1, [batch_size,784]).astype(np.float32)
    if np.random.rand(1)>1:
        batch_xs = norm_image
    x_bat = np.reshape(batch_xs,[-1,28,28])
    samples,_,_,loss_,cross_entropy_gen_,reconstruction_loss_ = sess.run([out_image,
                                                                        optim_recog,
                                                                          optim,
                                                                          loss,
                                                                          cross_entropy_gen,
                                                                          reconstruction_loss]
                                                                          ,feed_dict={x: batch_xs,y_:batch_ys})
    
    if step %50 ==0:
        print('step : '+str(step)+' loss : '+ str(loss_)\
              +' loss_reconstruction : '+ str(reconstruction_loss_)\
              +' loss_gen : '+ str(cross_entropy_gen_))
   
    if step % 2000 ==0:
        scipy.misc.imsave('./sample/%s_input.png'% (step),x_bat[1])    
        scipy.misc.imsave('./sample/%s_output.png'% (step),samples[1])
batch_xs, batch_ys = mnist.test.next_batch(30)
test_y = [[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]]
# feed_dict={x: batch_xs[:len(test_y)],y_:test_y}

In [ ]:
# def linear(input, output_dim, scope=None, stddev=0.01):
#     norm = tf.random_normal_initializer(stddev=stddev)
#     const = tf.constant_initializer(0.0)
#     with tf.variable_scope(scope or 'linear'):
#         w = tf.get_variable('w', [input.get_shape()[1], output_dim], initializer=norm)
#         b = tf.get_variable('b', [output_dim], initializer=const)
#         return tf.matmul(input, w) + b

In [ ]:
# Create the model
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

hidden  = tf.nn.relu(linear(y_,  1000, scope='c_h1', stddev=0.01))
a1 = tf.nn.relu(linear(hidden,  1000, scope='c_a1', stddev=0.01))
a2 = tf.nn.relu(linear(hidden,  1000, scope='c_a2', stddev=0.01))
a3 = tf.nn.relu(linear(hidden,  1000, scope='c_a3', stddev=0.01))
b1 = tf.nn.relu(linear(hidden,  1000, scope='c_b1', stddev=0.01))
b2 = tf.nn.relu(linear(hidden,  1000, scope='c_b2', stddev=0.01))
b3 = tf.nn.relu(linear(hidden,  1000, scope='c_b3', stddev=0.01))



h1 =linear(x,  1000, scope='c_layer1', stddev=0.01)
h1_c = tf.nn.relu(tf.mul(h1,a1)+b1)
h2 = tf.nn.relu(linear(h1_c, 1000, scope='c_layer2', stddev=0.01))
h2_c = tf.nn.relu(tf.mul(h2,a2)+b2)
h3 = tf.nn.relu(linear(h2_c, 1000, scope='c_layer3', stddev=0.01))
h3_c = tf.nn.relu(tf.mul(h3,a3)+b3)
h4 = linear(h3_c, 784, scope='c_layer4', stddev=0.01)
out_vecter = tf.sigmoid(h4)
out_image = tf.reshape(out_vecter,[-1,28,28])

def recognition_network(images,reuse = False):
    if reuse:
            tf.get_variable_scope().reuse_variables()
    output = tf.nn.relu(linear(images, 1000, scope='r_layer1', stddev=0.01))
    output = tf.nn.softmax(linear(output, 10, scope='r_layer2', stddev=0.01))
    return output
# Define loss and optimizer
y_clf = recognition_network(x)
y_clf_g = recognition_network(out_vecter,reuse =True)

cross_entropy_recog = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_clf, y_))
cross_entropy_gen = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_clf_g, y_)) 
# + \
# tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_clf_g, y_))
reconstruction_loss = tf.reduce_mean(tf.squared_difference(out_vecter,x))
loss = reconstruction_loss + 7.2*cross_entropy_gen
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
#train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
#t_vars = tf.trainable_variables()
t_vars = tf.trainable_variables()
contr_params = [var for var in t_vars if 'c_' in var.name]
recog_params = [var for var in t_vars if 'r_' in var.name]

optim = tf.train.AdamOptimizer(learning_rate=1e-3). \
        minimize(loss,var_list=contr_params) 
optim_recog = tf.train.AdamOptimizer(learning_rate=1e-3). \
        minimize(cross_entropy_recog,var_list=recog_params) 

session_conf = tf.ConfigProto(gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5),
                              allow_soft_placement=True,
                              log_device_placement=False)
sess = tf.InteractiveSession(config=session_conf)
# Train
tf.initialize_all_variables().run()
batch_size = 100
#pretrain recognition network
# for step in range(5001):
#     batch_xs, batch_ys = mnist.train.next_batch(batch_size)
#     _ = sess.run(optim_recog,feed_dict={x: batch_xs,y_:batch_ys})
for step in range(3001):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    norm_image = np.random.uniform(0, 1, [batch_size,784]).astype(np.float32)
    if np.random.rand(1)>1:
        batch_xs = norm_image
    x_bat = np.reshape(batch_xs,[-1,28,28])
    samples,_,_,loss_,cross_entropy_gen_,reconstruction_loss_ = sess.run([out_image,
                                                                        optim_recog,
                                                                          optim,
                                                                          loss,
                                                                          cross_entropy_gen,
                                                                          reconstruction_loss]
                                                                          ,feed_dict={x: batch_xs,y_:batch_ys})
    
    if step %50 ==0:
        print('step : '+str(step)+' loss : '+ str(loss_)\
              +' loss_reconstruction : '+ str(reconstruction_loss_)\
              +' loss_gen : '+ str(cross_entropy_gen_))
   
    if step % 2000 ==0:
        scipy.misc.imsave('./sample/%s_input.png'% (step),x_bat[1])    
        scipy.misc.imsave('./sample/%s_output.png'% (step),samples[1])
batch_xs, batch_ys = mnist.test.next_batch(30)
test_y = [[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]]
# feed_dict={x: batch_xs[:len(test_y)],y_:test_y}
# samples_test= sess.run(out_image, feed_dict=feed_dict)
# in_test= np.reshape(batch_xs[0:len(test_y)],[-1,28,28])
# for i in range(len(test_y)):
#     scipy.misc.imsave('./sample/%s_test_output.png'% (i),samples_test[i])
#     scipy.misc.imsave('./sample/%s_test_input.png'% (i),in_test[i])


In [ ]:
test_batch = batch_xs[-8:] 
test_image = []
for i in range(10):
    feed_dict={x: test_batch,y_:np.repeat([test_y[i]],8, axis=0)}
    samples_test= sess.run(out_image, feed_dict=feed_dict)
    test_image.append(samples_test)
test_image = np.array(test_image)    

In [ ]:
np.shape(test_image)

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline
def reshape_image(np_x,row,column):
    np_x = np_x.reshape((column,row,28,28))
    # split into 10 (1,10,28,28) images, concat along columns -> 1,10,28,280
    np_x = np.concatenate(np.split(np_x,column,axis=0),axis=3)
    # split into 10 (1,1,28,280) images, concat along rows -> 1,1,280,280
    np_x = np.concatenate(np.split(np_x,row,axis=1),axis=2)
    x_img = np.squeeze(np_x)
    return x_img

In [ ]:
img_test = reshape_image(test_image,8,10)
img_org = reshape_image(test_batch,8,1)

In [ ]:
f,axarr=plt.subplots(1,2,figsize=(8,3),sharey=True)
# samples
axarr[0].imshow(img_org,cmap=plt.cm.gray,interpolation='none')
axarr[0].set_title('original image')
# reconstruction
axarr[1].imshow(img_test,cmap=plt.cm.gray,interpolation='none')
axarr[1].set_title('Generated Images')

In [ ]:
#np.repeat([test_batch[i]],10, axis=0)